In [ ]:
# Processar arquivos que vieram do sistema do governo

import pandas as pd
import os
import glob

def process_file(input_file, estado='PB'):

    chunk_size = 100000  
    pb_count = 0
    pb_rows_list = []
    
    for chunk in pd.read_csv(input_file, sep=';', encoding='latin-1', header=None, chunksize=chunk_size, low_memory=False):
        
        chunk.columns = [f'Column{i+1}' for i in range(len(chunk.columns))]
        pb_rows = chunk[(chunk['Column20'] == estado) & (chunk['Column5'].notna()) & (chunk['Column5'] != '')]
        pb_rows = pb_rows[['Column5', 'Column12', 'Column20','Column21']]
        pb_count += len(pb_rows)
        pb_rows_list.append(pb_rows)

    if pb_rows_list:
        result_df = pd.concat(pb_rows_list, ignore_index=True)
        output_file = f"pb_results_{os.path.basename(input_file)}"
        result_df.to_csv(output_file, sep=';', encoding='latin-1', index=False)
        return result_df
    
    return None

files = glob.glob('./K32419/*.csv') # PASTA DE ENTRADA
all_results = []

for file_path in files:
    result = process_file(file_path)
    if result is not None:
        all_results.append(result)

if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df.to_csv('pb_results_all.csv', sep=';', encoding='latin-1', index=False)


In [ ]:
# Junção dos arquivos csvs
import pandas as pd

df = pd.read_csv('K324.csv', sep=';', encoding='latin-1')
df1 = pd.read_csv('K3241.csv', sep=';', encoding='latin-1')
df2 = pd.read_csv('K32412.csv', sep=';', encoding='latin-1')
df3 = pd.read_csv('K32413.csv', sep=';', encoding='latin-1')
df4 = pd.read_csv('K32414.csv', sep=';', encoding='latin-1')
df5 = pd.read_csv('K32415.csv', sep=';', encoding='latin-1')
df6 = pd.read_csv('K32416.csv', sep=';', encoding='latin-1')
df7 = pd.read_csv('K32417.csv', sep=';', encoding='latin-1')
df8 = pd.read_csv('K32418.csv', sep=';', encoding='latin-1')
df9 = pd.read_csv('K32419.csv', sep=';', encoding='latin-1')

df_concat = pd.concat([df, df1, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)

df_concat.to_csv('K324_concat.csv', sep=';', index=False, encoding='latin-1')

In [ ]:
# coloca legendas

import pandas as pd
import os

caminho_principal = 'K324_concat.csv'
caminho_legenda_atividade = './Cnaes/cnaes.csv'
caminho_legenda_cidade = './Municipios/municipios.csv'
caminho_saida = 'principal.csv' 

coluna_codigo_atividade_principal = 'Column12'
coluna_codigo_cidade_principal = 'Column21'

nomes_colunas_atividade = ['Codigo_Atividade_Str', 'Descricao_Atividade']
nomes_colunas_cidade = ['Codigo_Cidade_Str', 'Nome_Cidade']

DIGITOS_ATIVIDADE = 7
DIGITOS_CIDADE = 4 

df_principal = pd.read_csv(
    caminho_principal,
    sep=';',
    encoding='latin-1',
    dtype={
        coluna_codigo_atividade_principal: str,
        coluna_codigo_cidade_principal: str
    }
)
df_principal[coluna_codigo_atividade_principal] = df_principal[coluna_codigo_atividade_principal].astype(str).str.zfill(DIGITOS_ATIVIDADE)
df_principal[coluna_codigo_cidade_principal] = df_principal[coluna_codigo_cidade_principal].astype(str).str.zfill(DIGITOS_CIDADE)


df_atividades = pd.read_csv(
    caminho_legenda_atividade,
    sep=';',
    header=None,
    names=nomes_colunas_atividade,
    encoding='latin-1',
    dtype={nomes_colunas_atividade[0]: str}
)
df_atividades[nomes_colunas_atividade[0]] = df_atividades[nomes_colunas_atividade[0]].str.strip().str.strip('"')
df_atividades[nomes_colunas_atividade[1]] = df_atividades[nomes_colunas_atividade[1]].str.strip().str.strip('"')
df_atividades[nomes_colunas_atividade[0]] = df_atividades[nomes_colunas_atividade[0]].str.zfill(DIGITOS_ATIVIDADE)

df_cidades = pd.read_csv(
    caminho_legenda_cidade,
    sep=';',
    header=None,
    names=nomes_colunas_cidade,
    encoding='latin-1',
    dtype={nomes_colunas_cidade[0]: str}
)

df_cidades[nomes_colunas_cidade[0]] = df_cidades[nomes_colunas_cidade[0]].str.strip().str.strip('"')
df_cidades[nomes_colunas_cidade[1]] = df_cidades[nomes_colunas_cidade[1]].str.strip().str.strip('"')
df_cidades[nomes_colunas_cidade[0]] = df_cidades[nomes_colunas_cidade[0]].str.zfill(DIGITOS_CIDADE)

df_merged = pd.merge(
    df_principal,
    df_atividades,
    left_on=coluna_codigo_atividade_principal,
    right_on=nomes_colunas_atividade[0],
    how='left'
)

df_final = pd.merge(
    df_merged,
    df_cidades,
    left_on=coluna_codigo_cidade_principal,
    right_on=nomes_colunas_cidade[0],
    how='left'
)

df_final = df_final.rename(columns={
    nomes_colunas_atividade[1]: 'Descricao_Atividade_Nova',
    nomes_colunas_cidade[1]: 'Nome_Cidade_Nova'
})
df_final = df_final.drop(columns=[nomes_colunas_atividade[0], nomes_colunas_cidade[0]])

ativ_sem_match = df_final['Descricao_Atividade_Nova'].isnull()
num_ativ_sem_match = ativ_sem_match.sum()
if num_ativ_sem_match > 0:
    codigos_nao_encontrados_ativ = df_final.loc[ativ_sem_match, coluna_codigo_atividade_principal].unique()

cidade_sem_match = df_final['Nome_Cidade_Nova'].isnull()
num_cidade_sem_match = cidade_sem_match.sum()
if num_cidade_sem_match > 0:
    codigos_nao_encontrados_cid = df_final.loc[cidade_sem_match, coluna_codigo_cidade_principal].unique()

colunas_para_salvar = []
for col_original in df_principal.columns: 
    if col_original == coluna_codigo_atividade_principal:
        colunas_para_salvar.append('Descricao_Atividade_Nova') 
    elif col_original == coluna_codigo_cidade_principal:
        colunas_para_salvar.append('Nome_Cidade_Nova')       
    else:
        colunas_para_salvar.append(col_original)             

df_output = df_final[colunas_para_salvar]

df_output.to_csv(caminho_saida, sep=';', index=False, encoding='utf-8-sig')

In [ ]:
# fazer a planilha principal
import pandas as pd

caminho_arquivo_processado = 'principal.csv'
coluna_atividade = 'Descricao_Atividade_Nova'
caminho_contagem_saida = 'contagem_atividades.csv'

print(f"Lendo o arquivo processado: {caminho_arquivo_processado}")
try:
    # Use o encoding com o qual você salvou o arquivo anterior ('utf-8-sig')
    df = pd.read_csv(caminho_arquivo_processado, sep=';', encoding='utf-8-sig')
    print(f"Arquivo lido com sucesso. {len(df)} linhas.")
    # Verifica se a coluna de atividade existe
    if coluna_atividade not in df.columns:
        print(f"ERRO: A coluna '{coluna_atividade}' não foi encontrada no arquivo.")
        print(f"Colunas disponíveis: {df.columns.tolist()}")
        exit()
except FileNotFoundError:
    print(f"ERRO: Arquivo '{caminho_arquivo_processado}' não encontrado. Verifique o nome e o caminho.")
    exit()
except Exception as e:
    print(f"ERRO ao ler o arquivo CSV: {e}")
    exit()

# --- Análise de Frequência ---
print(f"\nAnalisando a frequência das atividades na coluna '{coluna_atividade}'...")

# Contar valores nulos (atividades que não foram encontradas no merge anterior)
nulos = df[coluna_atividade].isnull().sum()
if nulos > 0:
    print(f"Aviso: Foram encontrados {nulos} estabelecimentos sem descrição de atividade (valor nulo).")
    # Opção: Remover nulos antes de contar, ou eles serão ignorados por value_counts por padrão
    # df_filtrado = df.dropna(subset=[coluna_atividade])
    # contagem_atividades = df_filtrado[coluna_atividade].value_counts()
# Contar a frequência de cada descrição de atividade única
# value_counts() já ordena do mais frequente para o menos frequente
contagem_atividades = df[coluna_atividade].value_counts()

# --- Exibir Resultados ---
total_atividades_unicas = len(contagem_atividades)
print(f"\nTotal de tipos de atividades únicas encontradas: {total_atividades_unicas}")

# Mostrar as N atividades mais comuns
top_n = 20 # Você pode ajustar este número
print(f"\nAs {top_n} atividades mais comuns:")
print(contagem_atividades.head(top_n))

# Mostrar as N atividades menos comuns (opcional)
# bottom_n = 10
# print(f"\nAs {bottom_n} atividades menos comuns:")
# print(contagem_atividades.tail(bottom_n))

# --- Salvar Contagem (Opcional) ---
print(f"\nSalvando a contagem completa em '{caminho_contagem_saida}'...")
try:
    # Convertendo a Series (contagem) para um DataFrame para salvar com cabeçalhos
    df_contagem = contagem_atividades.reset_index()
    df_contagem.columns = ['Descricao_Atividade', 'Frequencia'] # Renomeia as colunas
    df_contagem.to_csv(caminho_contagem_saida, sep=';', index=False, encoding='utf-8-sig')
    print("Contagem salva com sucesso.")
except Exception as e:
    print(f"ERRO ao salvar o arquivo de contagem: {e}")

print("\nAnálise de frequência concluída.")

In [ ]:
# Filtrando a cidade
df = pd.read_csv('Paraiba_inteira.csv', sep=';', encoding='latin1')
df_joao_pessoa = df[df['Nome_Cidade_Nova'] == 'JOAO PESSOA']

In [ ]:
# CONTAGEM DE ATIVIDADES EM JOÃO PESSOA
contagem_atividades = df_joao_pessoa['Descricao_Atividade_Nova'].value_counts().reset_index()

contagem_atividades.columns = ['Descricao_Atividade', 'Frequencia']

contagem_atividades = contagem_atividades.sort_values('Frequencia', ascending=False)

contagem_atividades.to_csv('contagem_atividades_joao_pessoa.csv', sep=';', index=False, encoding='latin1')


In [7]:
# CATEGORIZAÇÃO DE ATIVIDADES
import pandas as pd
from unidecode import unidecode

def categorizar_atividade(descricao):
    if not isinstance(descricao, str):
        return "Não Informado"
    
    desc = unidecode(descricao.lower().strip()) # Normalizar e remover espaços extras

    # Atividade Não Informada
    if "atividade econonica nao informada" in desc or "atividade economica nao informada" in desc or not desc:
        return "Não Informado"

    # Serviços de Software
    if ("reparacao" in desc or "manutencao" in desc) and ("computador" in desc or "equipamentos perifericos" in desc):
        return "Serviços de Software"
    if ("reparacao" in desc or "manutencao" in desc) and "equipamentos de comunicacao" in desc:
        return "Serviços de Software"

    # Arte, Esporte e Lazer
    if desc == "agencias de viagens": # String exata
        return "Viagem"
    if "organizacao de excursoes" in desc:
        return "Viagem"
    if desc == "producao teatral":
        return "Arte"
    if desc == "Atividades de organizacoes associativas ligadas a cultura e a arte":
        return "Arte"
    if "comercio varejista de antiguidades" in desc or "comercio varejista de objetos de arte" in desc:
        return "Arte"
    if "atividades esportivas nao especificadas anteriormente" in desc:
         return "Esportes" 
    if "gestao de instalacoes de esportes" in desc:
        return "Esportes"
    if "comercio varejista de artigos esportivos" in desc:
        return "Esportes"
    if "comercio varejista de bicicletas e triciclos" in desc:
        return "Esportes"
    if "trens turisticos" in desc or "telefericos" in desc:
        return "Viagem"

    # Transportes, Logística, Locação de Veículos
    if ("locacao" in desc or "aluguel" in desc) and "automoveis" in desc and "sem condutor" in desc:
        return "Locação de Veículos"
    if "atividades auxiliares dos transportes terrestres" in desc:
        return "Transportes e Logística"
    if ("locacao" in desc or "aluguel" in desc) and "outros meios de transporte" in desc and "sem condutor" in desc:
        return "Locação de Veículos"
    if "armazens gerais" in desc: # Já tinha "armazenagem", mas para garantir
        return "Logística"
    if "servico de transporte de passageiros - locacao de automoveis com motorista" in desc:
        return "Locação de Veículos"
    if "atividades auxiliares dos transportes aereos" in desc:
        return "Locação de Veículos"
    if "operacao dos aeroportos e campos de aterrissagem" in desc:
        return "Transportes"
    if "agenciamento de cargas" in desc:
        return "Logística"
    if "comissaria de despachos" in desc or "despachantes aduaneiros" in desc:
        return "Logística"
    if "operador portuario" in desc:
        return "Logística"
    if "Fornecimento de alimentos preparados preponderantemente para empresas" in desc:
        return "Logística"
    if ("locacao" in desc or "aluguel" in desc) and "embarcacoes" in desc and "sem tripulacao" in desc and "recreativos" not in desc:
        return "Locação de Veículos"
    if "guarda-moveis" in desc:
        return "Logística"
    if "depositos de mercadorias para terceiros" in desc:
        return "Logística"

    # Agropecuária e Meio Ambiente
    if "atividades paisagisticas" in desc:
        return "Paisagismo"
    if "recuperacao de materiais plasticos" in desc:
        return "Meio Ambiente"
    if "recuperacao de sucatas de aluminio" in desc:
        return "Meio Ambiente"
    if "recuperacao de materiais metalicos" in desc: 
        return "Meio Ambiente"
    if "recuperacao de materiais nao especificados anteriormente" in desc: 
        return "Meio Ambiente"
    if "Coleta de residuos nao-perigosos" in desc: 
        return "Meio Ambiente"


    if "criacao de bovinos para corte" in desc or "criacao de peixes em agua doce" in desc or \
       "criacao de camaroes em agua salgada e salobra" in desc or "criacao de peixes ornamentais em agua doce" in desc or \
       "criacao de aves, exceto galinaceos" in desc or "criacao de suinos" in desc or "criacao de frangos para corte" in desc or \
       "criacao de outros animais nao especificados anteriormente" in desc or "ranicultura" in desc:
        return "Agropecuária"
    if "atividades relacionadas a esgoto" in desc:
        return "Meio Ambiente"
    if "poda de arvores para lavouras" in desc:
        return "Meio Ambiente"
    if "manejo de animais" in desc:
        return "Agropecuária"
    if "coleta de produtos nao-madeireiros" in desc:
        return "Meio Ambiente"
    if "tratamento e disposicao de residuos nao-perigosos" in desc or "tratamento e disposicao de residuos perigosos" in desc:
        return "Meio Ambiente"

    # Imobiliário
    if "compra e venda de imoveis proprios" in desc:
        return "Imobiliário"

    # Indústria
    if "confeccao, sob medida," in desc: # Já tinha "confeccao de", mas para garantir.
        return "Indústria"
    if "reforma de pneumaticos" in desc:
        return "Indústria"
    if ("decoracao" in desc or "lapidacao" in desc or "gravacao" in desc or "vitrificacao" in desc) and \
       ("ceramica" in desc or "louca" in desc or "vidro" in desc or "cristal" in desc):
        return "Indústria"
    if "geracao de energia eletrica" in desc or "transmissao de energia eletrica" in desc:
        return "Indústria"
    if "serrarias com desdobramento de madeira" in desc or "serrarias" in desc:
        return "Indústria"
    if "tratamentos termicos, acusticos ou de vibracao" in desc:
        return "Indústria"
    if "envasamento e empacotamento sob contrato" in desc:
        return "Indústria"
    if "corte e dobra de metais" in desc:
        return "Indústria"
    if "fundicao de ferro e aco" in desc:
        return "Indústria"
    if "torrefacao e moagem de cafe" in desc:
        return "Indústria"
    if "preparacao do leite" in desc:
        return "Indústria"
    if "lapidacao de gemas" in desc:
        return "Indústria"
    if "beneficiamento de arroz" in desc:
        return "Indústria"
    if "acabamento de calcados de couro sob contrato" in desc:
        return "Indústria"
    if "tratamento de madeira" in desc:
        return "Indústria"

    # Construção e Infraestrutura
    if "instalacao de maquinas e equipamentos industriais" in desc:
        return "Construção"
    if "instalacao de portas, janelas, tetos, divisorias" in desc:
        return "Construção"
    if ("aparelhamento de placas" in desc or "trabalhos em marmore" in desc or "trabalhos em granito" in desc or "trabalhos em ardosia" in desc):
        return "Construção"
    if "instalacoes de sistema de prevencao contra incendio" in desc:
        return "Construção"
    if "montagem e instalacao de sistemas e equipamentos de iluminacao e sinalizacao" in desc:
        return "Construção"
    if "preparacao de massa de concreto" in desc or "argamassa para construcao" in desc:
        return "Construção"
    if "restauracao e conservacao de lugares e predios historicos" in desc:
        return "Construção"
    if "pintura para sinalizacao em pistas" in desc:
        return "Construção"
    if "obras portuarias, maritimas e fluviais" in desc:
        return "Construção"
    if "montagem e desmontagem de andaimes" in desc:
        return "Construção"
    if "perfuracoes e sondagens" in desc:
        return "Construção"
    if "instalacao de equipamentos para orientacao a navegacao" in desc:
        return "Construção"
    if "preparacao de canteiro" in desc or "limpeza de terreno" in desc:
        return "Construção"
    
    if "Serviços de operacao e fornecimento de equipamentos para transporte e elevacao de cargas e pessoas para uso em obras" in desc:
        return "Construção"
    if "Manutenção de estacoes e redes de telecomunicacoes" in desc:
        return "Construção"

    # Serviços Pessoais e Domésticos
    if ("reparacao" in desc or "manutencao" in desc) and ("outros objetos e equipamentos pessoais e domesticos" in desc or "objetos e equipamentos pessoais e domesticos" in desc):
        return "Serviço de Reparação"
    if "reparacao de artigos do mobiliario" in desc:
        return "Serviço de Reparação"
    if "reparacao de joias" in desc:
        return "Serviço de Reparação"
    if "reparacao de relogios" in desc:
        return "Serviço de Reparação"
    if "reparacao de bicicletas" in desc or "reparacao de triciclos" in desc:
        return "Serviço de Reparação"
    if "alojamento de animais domesticos" in desc:
        return "Serviços de alojamento de animais domesticos"
    if "reparacao de calcados, bolsas e artigos de viagem" in desc:
        return "Serviço de Reparação"

    # Serviços de Consultoria
    if "gestao de ativos intangiveis" in desc:
        return "Serviços de Consultoria"
    if "auditoria e consultoria atuarial" in desc or "auditoria atuarial" in desc: # "auditoria atuarial" já existia
        return "Serviços de Consultoria"

    # Serviços Financeiros

    if "corretoras de titulos e valores mobiliarios" in desc:
        return "Serviços Financeiros"
    if "corretoras de contratos de mercadorias" in desc:
        return "Serviços Financeiros"
    if "sociedades de investimento" in desc:
        return "Serviços Financeiros"
    if "sociedades de capitalizacao" in desc:
        return "Serviços Financeiros"
    if "corretoras de cambio" in desc:
        return "Serviços Financeiros"
    if "instituicoes de intermediacao nao-monetaria" in desc:
        return "Serviços Financeiros"
    if "agencias de fomento" in desc:
        return "Serviços Financeiros"

    # Comércio - Veículos e Autopeças
    if "recondicionamento e recuperacao de motores" in desc:
        return "Veículos e Autopeças"
    if "comercio a varejo de pneumaticos e camaras-de-ar" in desc:
        return "Veículos e Autopeças"
    if "comercio varejista de embarcacoes e outros veiculos recreativos" in desc:
        return "Veículos e Autopeças"
    if "comercio varejista de lubrificantes" in desc:
        return "Veículos e Autopeças"
    if "comercio sob consignacao de veiculos automotores" in desc: 
        return "Veículos e Autopeças"

    # Farmácias e Drogarias (Mais específico primeiro)
    if "farmaceutic" in desc or "drogaria" in desc:
        return "Farmácias e Drogarias"

    # Saúde
    if "hospitalar" in desc or "pronto-socorro" in desc or "uti movel" in desc or \
       ("clinic" in desc and "laboratorios clinicos" not in desc and "veterinaria" not in desc) or \
       "atividade medica ambulatorial com recursos para realizacao de procedimentos cirurgicos" in desc or \
       "assistencia a paciente no domicilio" in desc: 
        return "Saúde"
    if "atividade medica ambulatorial com recursos para realizacao de exames complementares" in desc:
        return "Saúde"
    if "instituicoes de longa permanencia para idosos" in desc:
        return "Saúde"
    if "centros de apoio a pacientes com cancer e com aids" in desc:
        return "Saúde"
    if ("medica ambulatorial" in desc and "procedimentos cirurgicos" not in desc and "exames complementares" not in desc) or \
       "odontologic" in desc or "consultorios medicos" in desc:
        return "Saúde"
    if "laboratorio" in desc or "diagnostico por imagem" in desc or "analises clinicas" in desc or \
       "anatomia patologica" in desc or "citologica" in desc or "tomografia" in desc or "ressonancia magnetica" in desc or \
       "endoscopia" in desc or "ecg" in desc or "eeg" in desc:
        return "Saúde"
    if "psicologia" in desc or "psicanalise" in desc or "fisioterapia" in desc or \
       "fonoaudiologia" in desc or "terapia ocupacional" in desc or "acupuntura" in desc or \
       "praticas integrativas" in desc or "podologia" in desc or "nutricao" in desc:
        return "Saúde"
    if "profissionais da area de saude nao especificadas anteriormente" in desc or "atencao ambulatorial nao especificadas anteriormente" in desc:
        return "Saúde"
    if "comercio varejista de medicamentos veterinarios" in desc:
        return "Saúde"
    if "Atividades de apoio a gestao de saude" in desc or "Outras atividades de atenção à saúde humana não especificadas anteriormente" in desc or "Atividades funerarias e servicos relacionados nao especificados anteriormente" in desc or "Atividades de assistencia psicossocial e a saude a portadores de disturbios psiquicos, deficiencia mental e dependencia quimica e grupos similares nao" in desc:
        return "Saúde"

    # Serviços - Profissionais Diversos
    if "atividades profissionais, cientificas e tecnicas" in desc:
        return "Serviço Profissional Diverso"
    if "pesquisa e desenvolvimento experimental" in desc:
        return "Serviço Profissional Diverso"
    if "testes e analises tecnicas" in desc:
        return "Serviço Profissional Diverso"
    if "restauracao de obras-de-arte" in desc:
        return "Serviço Profissional Diverso"
    if "edicao integrada a impressao de cadastros" in desc or "edicao integrada a impressao de listas" in desc:
        return "Serviço Profissional Diverso"
    if "agencias de noticias" in desc:
        return "Serviço Profissional Diverso"
    if ("impressao de livros" in desc or "impressao de revistas" in desc or "publicacoes periodicas" in desc) and "edicao integrada" not in desc : # Evitar duplicar com "edição integrada"
        return "Serviço Profissional Diverso"
    if "edicao integrada a impressao de livros" in desc:
        return "Serviço Profissional Diverso"
    if "edicao integrada a impressao de revistas" in desc:
        return "Serviço Profissional Diverso"
    if "edicao integrada a impressao de jornais diarios" in desc:
        return "Serviço Profissional Diverso"
    if "instalacao de paineis publicitario" in desc:
        return "Serviço Profissional Diverso"
    if "pesquisas de mercado" in desc or "opiniao publica" in desc:
        return "Serviço Profissional Diverso"
    if "atividades associativas profissionais" in desc:
        return "Serviço Profissional Diverso"
    if "atividades associativas nao especificadas anteriormente" in desc: # "associacoes" já existe
        return "Serviço Profissional Diverso"
    if "arrendamento mercantil" in desc:
        return "Serviço Profissional Diverso"
    if "sociedades de participacao" in desc:
        return "Serviço Profissional Diverso"
    if "emissao de vales-alimentacao" in desc or "emissao de vales-transporte" in desc:
        return "Serviço Profissional Diverso"
    if "planos de auxilio-funeral" in desc:
        return "Serviço Profissional Diverso"
    if "gestao e manutencao de cemiterios" in desc:
        return "Serviço Profissional Diverso"
    if "instalacao de outros equipamentos nao especificados anteriormente" in desc:
        return "Serviço Profissional Diverso"
    if "medicao de consumo de energia eletrica, gas e agua" in desc:
        return "Serviço Profissional Diverso"
    if "organismos internacionais" in desc:
        return "Serviço Profissional Diverso"
    if desc == "orfanatos":
        return "Serviço Profissional Diverso"
    if "arquitetura" in desc or "engenharia" in desc or "urbanizacao" in desc or "desenho tecnico" in desc \
        or "cartografia" in desc or "topografia" in desc or "geodesia" in desc:
        return "Serviço Profissional Diverso"
    if "contabilidade" in desc or ("consultoria em gestao" in desc and "ativos intangiveis" not in desc) or \
        "auditoria contabel" in desc:
        return "Serviço Profissional Diverso"
    if "advocacia" in desc or "advocatícios" in desc:
        return "Serviço Profissional Diverso"
    if "marketing" in desc or ("publicidade" in desc and "paineis" not in desc) or \
       "promocao de vendas" in desc or ("agencias de publicidade" in desc and "noticias" not in desc) or \
       ("agenciamento de espacos para publicidade" in desc): 
        return "Serviço Profissional Diverso"
    if "traducao" in desc or "interpretacao" in desc or \
       ("design" in desc and "web design" not in desc and "interiores" not in desc and "produto" not in desc) or \
       ("producao de fotografias" in desc and "aerea e submarina" not in desc) or \
       "pericia tecnica" in desc or "agenciamento de profissionais" in desc or \
       ("consultoria tecnica especifica" in desc and "gestao empresarial" not in desc and "tecnologia" not in desc and "publicidade" not in desc and "atuarial" not in desc):
        return "Serviço Profissional Diverso"
    if "design de interiores" in desc or "design de produto" in desc:
        return "Serviço Profissional Diverso"
    if ("servicos" in desc and "domesticos" not in desc and "funerarias" not in desc and "alimentacao" not in desc and "transporte" not in desc and "escritorio" not in desc) or \
       ("atividades de" in desc and "ensino" not in desc and "saude" not in desc and "associativas" not in desc and "esportivas" not in desc and "profissionais" not in desc and "radio" not in desc and "televisao" not in desc) or \
       ("manutencao e reparacao" in desc and "computador" not in desc and "equipamentos de comunicacao" not in desc and "objetos pessoais" not in desc and "veiculos" not in desc and "motocicletas" not in desc) or \
       ("aluguel de" in desc and "automoveis" not in desc and "meios de transporte" not in desc and "imoveis" not in desc and "embarcacoes" not in desc):
        return "Serviço Profissional Diverso"

    # Serviços - TI e Desenvolvimento de Software (reparos já cobertos acima)
    if ("informatica" in desc and "treinamento" not in desc) or "software" in desc or "programas de computador" in desc or \
       "tecnologia da informacao" in desc or "internet" in desc or "web design" in desc or \
       "portais" in desc or "provedores de conteudo" in desc or "consultoria em tecnologia" in desc or \
       "suporte tecnico" in desc or "tratamento de dados" in desc or "hospedagem na internet" in desc or \
       "servicos de comunicacao multimedia - scm" in desc or ("redes de comunicacoes" in desc and "construcao" not in desc) or \
       "comercio varejista de artigos fotograficos e para filmagem" in desc or \
       "comercio varejista de discos, cds, dvds e fitas" in desc or \
       "comercio varejista especializado de eletrodomesticos e equipamentos de audio e video" in desc:
        return "Eletronicos"

    # Educação 
    if "educacao infantil" in desc or "creche" in desc or "pre-escola" in desc or \
       "ensino fundamental" in desc or "ensino medio" in desc:
        return "Educação"
    if "educacao superior" in desc or "universidade" in desc or "faculdade" in desc:
        return "Educação"
    if "comercio varejista de artigos de papelaria" in desc:
        return "Educação"
    if "treinamento" in desc or "cursos preparatorios" in desc or "ensino de idiomas" in desc or \
       "ensino de musica" in desc or "ensino de arte" in desc or "ensino de danca" in desc or \
       "ensino de esportes" in desc or "formacao de condutores" in desc or "cursos de pilotagem" in desc or \
       "educacao profissional" in desc or ("outras atividades de ensino" in desc and "superior" not in desc and "fundamental" not in desc and "medio" not in desc):
        return "Educação"
    if "bibliotecas" in desc or "editoras" in desc or ("edicao de livros" in desc and "integrada" not in desc) or \
       ("edicao de jornais" in desc and "integrada" not in desc) or ("edicao de revistas" in desc and "integrada" not in desc) or \
       "livraria" in desc or ("edicao de cadastros, listas" in desc and "integrada" not in desc) or "comercio varejista de livros" in desc:
        return "Educação"

    # Indústria (Fabricação/Produção) - (vários específicos já cobertos acima)
    if "fabricacao" in desc or ("confeccao de" in desc and "sob medida" not in desc) or \
       ("producao" in desc and "agricola" not in desc and "pecuaria" not in desc and "florestal" not in desc and "extracao mineral" not in desc and "pesca" not in desc and "eventos" not in desc and "fotografia" not in desc and "filmes" not in desc and "musical" not in desc and "teatral" not in desc) or \
       "industrial" in desc or "metalurgia" in desc or "siderurgica" in desc or ("moagem" in desc and "cafe" not in desc) or \
       "abate" in desc or "usina" in desc or "fiacao" in desc or "tecelagem" in desc or "estamparia" in desc or \
       ("serralheria" in desc and "comercio" not in desc and "desdobramento" not in desc) or \
       ("marcenaria" in desc and "comercio" not in desc) or \
       "panificacao industrial" in desc or ("impressao de material" in desc and "livros" not in desc and "revistas" not in desc and "jornais" not in desc and "cadastros" not in desc) or \
       ("faccao de" in desc):
        return "Indústria"
    
    # Construção
    if ("construcao de" in desc and "maquinas" not in desc) or ("edificios" in desc and "pintura" not in desc and "limpeza" not in desc) or \
       ("obras de" in desc and "portuarias" not in desc and "maritimas" not in desc and "fluviais" not in desc and "arte" not in desc) or \
       "instalacao e manutencao eletrica" in desc or "alvenaria" in desc or "pintura de edificios" in desc or \
       ("montagem de estruturas" in desc and "andaimes" not in desc) or \
       "terraplenagem" in desc or "pavimentacao" in desc or "sistemas centrais de ar condicionado" in desc or "elevadores" in desc or \
       "instalacoes hidraulicas" in desc or "gesso e estuque" in desc or "impermeabilizacao" in desc or \
        "demolicao" in desc or "fundacoes" in desc or "revestimentos" in desc or \
        ("redes de telecomunicacoes" in desc and "construcao" in desc) or ("redes de distribuicao de energia" in desc and "construcao" in desc) or \
        "administracao de obras" in desc:
        return "Construção"
    if "materiais de construcao" in desc or ("ferragens e ferramentas" in desc and "comercio varejista" in desc) or \
       ("material eletrico" in desc and "varejista" in desc) or \
       ("tintas e materiais para pintura" in desc and "varejista" in desc) or ("vidros" in desc and "varejista" in desc) or \
       ("materiais hidraulicos" in desc and "varejista" in desc) or \
       ("madeira e artefatos" in desc and "varejista" in desc and "moveis" not in desc) or \
       ("pedras para revestimento" in desc and "varejista" in desc) or ("cal, areia, pedra britada, tijolos e telhas" in desc and "varejista" in desc):
        return "Construção"

    # Veículos e Autopeças
    if (("pecas e acessorios" in desc or "pneumaticos e camaras-de-ar" in desc or "lubrificantes" in desc or "combustiveis para veiculos automotores" in desc) and \
        ("veiculos automotores" in desc or "motocicletas" in desc or "motonetas" in desc or "automoveis" in desc or "caminhoes" in desc or "onibus" in desc)) or \
       (("automoveis" in desc or "motocicletas e motonetas" in desc or "caminhoes" in desc or "onibus" in desc) and \
        ("varejo" in desc or "atacado" in desc) and "consignacao" not in desc): # consignacao já coberto
        return "Veículos e Autopeças"
    if "servicos de lavagem, lubrificacao e polimento de veiculos automotores" in desc or \
       "servicos de manutencao e reparacao mecanica de veiculos automotores" in desc or \
       "servicos de lanternagem ou funilaria e pintura de veiculos automotores" in desc or \
       "instalacao, manutencao e reparacao de acessorios para veiculos automotores" in desc or \
       "manutencao e reparacao de motocicletas e motonetas" in desc or \
       "servicos de borracharia para veiculos automotores" in desc or \
       "servicos de alinhamento e balanceamento de veiculos automotores" in desc or \
       "reboque de veiculos" in desc or \
       "outros transportes rodoviarios de passageiros nao especificados anteriormente" in desc or \
       "transporte maritimo de cabotagem - passageiros" in desc or \
       "transporte por navegacao interior de carga, municipal, exceto travessia" in desc or \
       "comercio sob consignacao de motocicletas e motonetas" in desc or \
       "manutencao e reparacao de baterias e acumuladores eletricos, exceto para veiculos" in desc or \
       "outros transportes aquaviarios nao especificados anteriormente" in desc or \
       "locacao de embarcacoes sem tripulacao, exceto para fins recreativos" in desc or \
       "transporte maritimo de cabotagem - carga" in desc:
        return "Veículos e Autopeças"

    # Supermercados
    if ("minimercados, mercearias e armazens" in desc or "hipermercados" in desc or "supermercados" in desc) and "produtos alimenticios" in desc:
        return "Supermercados"
    if "comercio varejista de mercadorias em geral, com predominancia de produtos alimenticios" in desc and \
       ("minimercado" in desc or "mercearia" in desc or "armazem" in desc or "hipermercado" in desc or "supermercado" in desc):
        return "Supermercados"
    if "lojas de conveniencia" in desc:
        return "Supermercados"
    if "Peixaria" in desc:
        return "Supermercados"
    if "comercio varejista de carnes" in desc or "acougues" in desc:
        return "Supermercados"
    if "comercio varejista de hortifrutigranjeiros" in desc:
        return "Supermercados"
    if "comercio varejista de jornais e revistas" in desc:
        return "Supermercados"
    if "comercio varejista de laticinios e frios" in desc:
        return "Supermercados"
    if "supermercado" in desc:
        return "Supermercados"
    if "lojas de conveniencia" in desc:
        return "Supermercados"
    
    if "servicos ambulantes de alimentacao" in desc:
        return "FastFood"

    # Comércio - Lojas de Roupa e Acessórios
    if "vestuario e acessorios" in desc or ("calcados" in desc and "varejista" in desc and "reparacao" not in desc and "acabamento" not in desc) or \
       ("artigos de armarinho" in desc and "varejista" in desc) or ("artigos de viagem" in desc and "varejista" in desc and "reparacao" not in desc) or \
       ("joalheria" in desc and ("comercio" in desc or "varejista" in desc) and "reparacao" not in desc) or \
       ("relojoaria" in desc and ("comercio" in desc or "varejista" in desc) and "reparacao" not in desc) or \
       ("bijuterias e artesanatos" in desc and "varejista" in desc) or \
       ("cama, mesa e banho" in desc and "varejista" in desc) or ("tecidos" in desc and "varejista" in desc) or \
       ("artigos de optica" in desc and "comercio varejista" in desc) or \
       "confeccao de pecas de vestuario, exceto roupas intimas e as confeccionadas sob medida" in desc or \
       "confeccao de roupas profissionais, exceto sob medida" in desc:
        return "Lojas de Roupa e Acessórios"

    # Alimentação
    if "restaurantes e similares" in desc:
        return "Restaurantes"
    if "Fornecimento de alimentos preparados preponderantemente para consumo domiciliar" in desc:
        return "Restaurantes"
    if "lanchonetes, casas de cha, de sucos e similares" in desc:
        return "Lanchonetes"
    if "comercio varejista de doces, balas, bombons e semelhantes" in desc:
        return "Lanchonetes"
    if "Cantinas - serviços de alimentação privativos" in desc:
        return "Lanchonetes"
    if "bares e outros estabelecimentos especializados em servir bebidas" in desc or "comercio varejista de bebidas" in desc:
        return "Bares"
    if "padaria e confeitaria" in desc and "industrial" not in desc:
        return "Padaria"
    if "fornecimento de alimentos preparados" in desc or "servicos de alimentacao para eventos e recepcoes - bufe" in desc or "cantinas - servicos de alimentacao privativos" in desc:
        return "Delivery"
    if "Serviços de alimentação para eventos e recepções - bufê" in desc:
        return "Bufê"

    if "hoteis" in desc or "pousadas" in desc or ("alojamento" in desc and "animais" not in desc) or "moteis" in desc or "apart-hoteis" in desc or "albergues" in desc or "pensoes (alojamento)" in desc:
        return "Hospedagem"

    # Transportes e Logística (vários específicos já cobertos)
    if ("transporte rodoviario" in desc and "auxiliares" not in desc) or \
       ("transporte de carga" in desc and "agenciamento" not in desc and "navegacao" not in desc) or \
       "logistica" in desc or ("armazenagem" in desc and "armazens gerais" not in desc) or \
       "distribuicao" in desc or "mudancas" in desc or "taxi" in desc or \
       ("entrega rapida" in desc and "alimentos" not in desc and "refeicoes" not in desc) or \
       "malote" in desc or "correio nacional" in desc or "transporte escolar" in desc or \
       ("transporte aereo" in desc and "auxiliares" not in desc) or \
       ("transporte aquaviario" in desc and "outros" not in desc and "navegacao" not in desc) or \
       "transporte ferroviario" in desc or ("operador de transporte" in desc and "portuario" not in desc) or \
       "carga e descarga" in desc or "terminais rodoviarios e ferroviarios" in desc or "estacionamento de veiculos" in desc:
        return "Transportes"

    # Arte, Entretenimento e Lazer (vários específicos já cobertos)
    if "cinema" in desc or ("cinematografica" in desc and "producao" not in desc and "distribuicao" not in desc) or \
       ("teatro" in desc and "producao" not in desc) or "show" in desc or \
       ("eventos e festas" in desc and "alimentacao" not in desc and "filmagem" not in desc) or \
       "producao musical" in desc or "estudios de gravacao" in desc or "lazer" in desc or "recreacao" in desc or \
       "jogos eletronicos" in desc or "parques de diversao" in desc or \
       ("agencias de viagem" in desc and "noticias" not in desc) or \
       "turismo" in desc or "operadores turisticos" in desc or "artes cenicas" in desc or "espetaculos" in desc or \
       ("exposicoes" in desc and "organizacao" not in desc) or \
       "discotecas" in desc or "danceterias" in desc or "casas de festas" in desc or "casas de bingo" in desc or \
       "boliches" in desc or "exploracao de jogos" in desc or "producao e promocao de eventos esportivos" in desc or \
       "comercio varejista de brinquedos e artigos recreativos" in desc or \
       "comercio varejista de fogos de artificio e artigos pirotecnicos" in desc or \
       "comercio varejista especializado de instrumentos musicais e acessorios" in desc or \
       "lojas de departamentos ou magazines, exceto lojas francas" in desc:
        return "Entretenimento"
    
    if desc == "Atividades relacionadas a televisao por assinatura, exceto programadoras":
        return "Entretenimento"
    if "organizacao de feiras, congressos, exposicoes e festas" in desc: # Caso genérico de organização de eventos
        return "Arte"


    # Agropecuária e Meio Ambiente (vários específicos já cobertos)
    if "agropecuaria" in desc or "agricola" in desc or "pecuaria" in desc or "fazendas" in desc or \
       "cooperativa agricola" in desc or "agroindustria" in desc or "floriculturas" in desc or \
       ("criacao de animais" in desc and "bovinos" not in desc and "peixes" not in desc and "camaroes" not in desc and "aves" not in desc and "suinos" not in desc and "frangos" not in desc) or \
       "pesca" in desc or "cultivo" in desc or ("extracao" in desc and "mineral" not in desc) or \
       "servicos ambientais" in desc or ("reciclagem" in desc and "materiais" not in desc) or \
       ("coleta de residuos" in desc and "nao-perigosos" not in desc and "perigosos" not in desc) or \
       "meio ambiente" in desc or "agronomia" in desc or "veterinaria" in desc or \
       "imunizacao e controle de pragas urbanas" in desc or \
       ("plantas e flores naturais" in desc and "comercio" in desc) or "horticultura" in desc or "apicultura" in desc:
        return "Agropecuária"

    # Serviços Pessoais e Domésticos (vários específicos já cobertos)
    if "cabeleireiros, manicure e pedicure" in desc or "estetica e outros servicos de cuidados com a beleza" in desc or \
       "lavanderias" in desc or "chaveiros" in desc or \
       ("reparacao e manutencao de equipamentos eletroeletronicos de uso pessoal e domestico" in desc and "outros objetos" not in desc) or \
       "servicos domesticos" in desc or "limpeza em predios e em domicilios" in desc or \
       "higiene e embelezamento de animais domesticos" in desc or "tatuagem e colocacao de piercing" in desc or \
       "servicos de funerarias" in desc or "agencias matrimoniais" in desc or "toalheiros" in desc or \
       "comercio varejista de artigos de colchoaria" in desc or "comercio varejista de artigos de iluminacao" in desc or \
       "comercio varejista de artigos de tapecaria, cortinas e persianas" in desc or \
       "comercio varejista de equipamentos para escritorio" in desc or "comercio varejista de moveis" in desc or \
       "comercio varejista de outros artigos de uso pessoal e domestico nao especificados anteriormente" in desc or \
       "comercio varejista de outros artigos usados" in desc or \
       "comercio varejista de produtos saneantes domissanitarios" in desc or "tabacaria" in desc:
        return "Serviços Pessoais e Domésticos"
    if ("animais vivos e de artigos e alimentos para animais de estimacao" in desc and "comercio varejista" in desc and "alojamento" not in desc) or \
       ("comercio varejista de artigos medicos e ortopedicos" in desc):
        return "Serviços Pessoais e Domésticos"

    # Serviços Financeiros e Seguros (vários específicos já cobertos)
    if "bancos" in desc or ("financeiras" in desc and "instituicoes de intermediacao nao-monetaria" not in desc) or \
       ("seguros" in desc and "planos de auxilio-funeral" not in desc) or "corretor de seguros" in desc or \
       "previdencia" in desc or ("investimentos" in desc and "sociedades de investimento" not in desc) or \
       "credito" in desc or "cobrancas e informacoes cadastrais" in desc or \
       "correspondentes de instituicoes financeiras" in desc or "fomento mercantil - factoring" in desc or \
       "consorcios" in desc or "planos de saude" in desc or "caixas economicas" in desc or "casas lotericas" in desc or \
       "administracao de cartoes de credito" in desc or "leiloeiros independentes" in desc:
        return "Serviços Financeiros"

    # Imobiliário (compra e venda já coberto)
    if ("imobiliari" in desc and "empreendimentos" not in desc) or "corretagem na compra e venda e avaliacao de imoveis" in desc or \
       "aluguel de imoveis" in desc or "incorporacao de empreendimentos" in desc or "loteamento de imoveis" in desc or \
       "gestao e administracao da propriedade imobiliaria" in desc or "condominios prediais" in desc:
        return "Imobiliário"
    
    # Comércio Atacadista
    if "comercio atacadista" in desc or "atacado de" in desc:
        return "Atacadista"
    if "representantes comerciais e agentes do comercio" in desc:
        return "Atacadista"


    # Fitness e Bem-Estar (não médico)
    if "condicionamento fisico" in desc or "academia" in desc or "pilates" in desc or "yoga" in desc or \
       ("esportivos" in desc and ("artigos" not in desc and "clubes" not in desc and "eventos" not in desc and "instalacoes" not in desc and "atividades nao especificadas" not in desc)):
        return "Fitness"

    # Telecomunicações (reparo de equipamentos e TV por assinatura já cobertos)
    if ("telecomunicacoes" in desc and "redes" not in desc) or "telefonia" in desc or "scm" in desc or "srtt" in desc or "sme" in desc or "voip" in desc or \
       ("operadoras de televisao por assinatura" in desc and "atividades relacionadas" not in desc) or \
       "atividades de radio" in desc or "atividades de televisao aberta" in desc:
        return "Telecomunicações"

    # Serviços Administrativos e de Apoio a Empresas
    if "servicos combinados de escritorio e apoio administrativo" in desc or \
       "preparacao de documentos e servicos especializados de apoio administrativo" in desc or \
       "fotocopias" in desc or "secretariado" in desc or "selecao e agenciamento de mao-de-obra" in desc or \
       "locacao de mao-de-obra temporaria" in desc or "fornecimento e gestao de recursos humanos" in desc or \
       "servicos de apoio a edificios" in desc or "teleatendimento" in desc:
        return "Serviços Administrativos"

    # Organizações Associativas e Religiosas (profissionais e não especificadas já cobertas)
    if ("associacoes" in desc and "profissionais" not in desc and "nao especificadas" not in desc) or \
        "sindicatos" in desc or "organizacoes religiosas" in desc or "filosoficas" in desc or \
       "clubes sociais, esportivos e similares" in desc or "organizacoes politicas" in desc:
        return "Organizações Associativas e Religiosas"

    # Administração Pública, Defesa e Seguridade Social
    if "administracao publica" in desc or "defesa" in desc or "seguranca e ordem publica" in desc or \
       "justica" in desc or "seguridade social obrigatoria" in desc or "regulacao das atividades" in desc or \
       "atividades de fiscalizacao profissional" in desc:
        return "Administração Pública"

    # Comércio Varejista - Outros (mais genérico, por último entre os comércios)
    if "comercio varejista" in desc or "varejo de" in desc or "lojas de departamento" in desc or "lojas de variedades" in desc or "tabacaria" in desc:
        return "Comércio Varejista"

    return "Não Classificado"


input_filename = 'Joao_Pessoa.csv'
output_filename = 'atividades_categorizadas_jp.csv'

try:
    df = pd.read_csv(input_filename, delimiter=';', encoding='utf-8')
    print(f"Arquivo '{input_filename}' lido com sucesso (delimitador=';', encoding='utf-8').")
except FileNotFoundError:
    print(f"ERRO: Arquivo '{input_filename}' não encontrado.")
    print("Por favor, certifique-se de que o arquivo CSV com as atividades está na mesma pasta que este script.")
    exit()
except UnicodeDecodeError:
    print(f"ERRO de decodificação ao ler '{input_filename}' com UTF-8. Tentando com Latin-1...")
    try:
        df = pd.read_csv(input_filename, delimiter=';', encoding='latin1')
        print(f"Arquivo '{input_filename}' lido com sucesso (delimitador=';', encoding='latin1').")
    except Exception as e_latin1:
        print(f"ERRO ao ler o arquivo CSV '{input_filename}' também com Latin-1: {e_latin1}")
        exit()
except Exception as e:
    print(f"ERRO geral ao ler o arquivo CSV '{input_filename}': {e}")
    exit()

# Verificar se a coluna 'Descricao_Atividade' existe
if 'Descricao_Atividade_Nova' not in df.columns:
    # Tenta encontrar uma coluna similar ignorando case e espaços extras
    col_candidatas = [col for col in df.columns if unidecode(col.strip().lower()) == "Descricao_Atividade_Nova"]
    if col_candidatas:
        nome_coluna_atividade = col_candidatas[0]
        print(f"AVISO: Coluna 'Descricao_Atividade_Nova' não encontrada exatamente. Usando '{nome_coluna_atividade}' como coluna de atividades.")
    else:
        print(f"ERRO: A coluna 'Descricao_Atividade_Nova' (ou similar) não foi encontrada no arquivo '{input_filename}'.")
        print(f"Colunas encontradas: {list(df.columns)}")
        print("Verifique o cabeçalho do seu CSV.")
        exit()
else:
    nome_coluna_atividade = 'Descricao_Atividade_Nova'


# Aplicar a função de categorização para criar a nova coluna
df['Grupo_Atividade'] = df[nome_coluna_atividade].apply(categorizar_atividade)
print("Categorização das atividades concluída.")

# Salvar o DataFrame resultante em um novo arquivo CSV
df.to_csv(output_filename, index=False, encoding='utf-8-sig', sep=';') # Salvar com ; também
print(f"Arquivo '{output_filename}' gerado com sucesso (delimitador=';', encoding='utf-8-sig')!")

print("\nAmostra do resultado (primeiras 20 linhas):")
print(df.head(20).to_string())

print("\nContagem de atividades por grupo:")
print(df['Grupo_Atividade'].value_counts().to_string())

print("\nAtividades ainda não classificadas (se houver):")
nao_classificadas = df[df['Grupo_Atividade'] == 'Não Classificado']
if not nao_classificadas.empty:
    print(nao_classificadas[[nome_coluna_atividade, 'Grupo_Atividade']].to_string())
else:
    print("Todas as atividades foram classificadas!")


Arquivo 'Joao_Pessoa.csv' lido com sucesso (delimitador=';', encoding='utf-8').
Categorização das atividades concluída.
Arquivo 'atividades_categorizadas_jp.csv' gerado com sucesso (delimitador=';', encoding='utf-8-sig')!

Amostra do resultado (primeiras 20 linhas):
                                             Column5                                                                                                        Descricao_Atividade_Nova Column20 Nome_Cidade_Nova                 Grupo_Atividade
0                                             LE PUB                                            Bares e outros estabelecimentos especializados em servir bebidas, com entretenimento       PB      JOAO PESSOA                           Bares
1   JD MARKETING, PRODUCOES E EDICOES DE AUDIO VIDEO               Atividades de pós-produção cinematográfica, de vídeos e de programas de televisão não especificadas anteriormente       PB      JOAO PESSOA                       Indústria
2               

In [8]:
df = pd.read_csv('atividades_categorizadas_jp.csv', sep=';', encoding='utf-8-sig')

grupo_counts = df['Grupo_Atividade'].value_counts().reset_index()
grupo_counts.columns = ['Grupo_Atividade', 'Quantidade']

# Display the results
print(f"Total de grupos de atividades: {len(grupo_counts)}")
print("\nGrupos de atividades e suas quantidades:")
print(grupo_counts.to_string(index=False))

# List just the unique values
print("\nLista de valores únicos em Grupo_Atividade:")
print(df['Grupo_Atividade'].unique())

Total de grupos de atividades: 43

Grupos de atividades e suas quantidades:
                             Grupo_Atividade  Quantidade
                Serviço Profissional Diverso       24470
                 Lojas de Roupa e Acessórios       14823
                                  Construção       10922
              Serviços Pessoais e Domésticos        8300
                                    Educação        7421
                                 Imobiliário        7296
                                       Saúde        6909
                               Supermercados        6463
                                   Indústria        6405
                                 Lanchonetes        4691
                        Veículos e Autopeças        4165
                                Restaurantes        4002
                          Comércio Varejista        3920
                                  Atacadista        3862
                                 Eletronicos        3659
            